## Dashboard

In this Notebook, we interact with the Experiment Manager to configure, setup and run experiments.

### Import dependencies, initialise configs

In [1]:
import sys
sys.path.append('../')
%config Completer.use_jedi = False # to avoid autocomplete errors in Jupyter server
from ipywidgets import GridspecLayout, GridBox, Layout, Output
import dashboard_ui

dashboard_ui = dashboard_ui.DashboardUI()

### Choose Model, Dataset and Fusion Algorithm

#### Provide Data Handler
- Only if you wish to use a Custom Dataset
- Choose Yes in the `Custom Dataset?` option below

Populate and then run the cell below to save the provided Data Handler class to file.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%%writefile custom_data_handler.py
### YOUR DATAHANDLER code goes below

import logging

import numpy as np

from ibmfl.data.data_handler import DataHandler
from ibmfl.util.datasets import load_mnist

logger = logging.getLogger(__name__)


class MnistKerasDataHandler(DataHandler):
    """
    Data handler for MNIST dataset.
    """

    def __init__(self, data_config=None, channels_first=False):
        super().__init__()

        self.file_name = None
        if data_config is not None:
            # Ensure your data files are either npz or csv
            if 'npz_file' in data_config:
                self.file_name = data_config['npz_file']
            elif 'txt_file' in data_config:
                self.file_name = data_config['txt_file']
        self.channels_first = channels_first

        # load the datasets
        (self.x_train, self.y_train), (self.x_test, self.y_test) = self.load_dataset()

        # pre-process the datasets
        self.preprocess()

    def get_data(self):
        """
        Gets pre-process mnist training and testing data.

        :return: the training and testing data.
        :rtype: `tuple`
        """
        return (self.x_train, self.y_train), (self.x_test, self.y_test)

    def load_dataset(self, nb_points=500):
        """
        Loads the training and testing datasets from a given local path. \
        If no local path is provided, it will download the original MNIST \
        dataset online, and reduce the dataset size to contain \
        500 data points per training and testing dataset. \
        Because this method \
        is for testing it takes as input the number of datapoints, nb_points, \
        to be included in the training and testing set.

        :param nb_points: Number of data points to be included in each set if
        no local dataset is provided.
        :type nb_points: `int`
        :return: training and testing datasets
        :rtype: `tuple`
        """
        if self.file_name is None:
            (x_train, y_train), (x_test, y_test) = load_mnist()
            # Reduce datapoints to make test faster
            x_train = x_train[:nb_points]
            y_train = y_train[:nb_points]
            x_test = x_test[:nb_points]
            y_test = y_test[:nb_points]
        else:
            try:
                logger.info('Loaded training data from ' + str(self.file_name))
                data_train = np.load(self.file_name)
                x_train = data_train['x_train']
                y_train = data_train['y_train']
                x_test = data_train['x_test']
                y_test = data_train['y_test']
            except Exception:
                raise IOError('Unable to load training data from path '
                              'provided in config file: ' +
                              self.file_name)
        return (x_train, y_train), (x_test, y_test)

    def preprocess(self):
        """
        Preprocesses the training and testing dataset, \
        e.g., reshape the images according to self.channels_first; \
        convert the labels to binary class matrices.

        :return: None
        """
        num_classes = 10
        img_rows, img_cols = 28, 28

        if self.channels_first:
            self.x_train = self.x_train.reshape(self.x_train.shape[0], 1, img_rows, img_cols)
            self.x_test = self.x_test.reshape(self.x_test.shape[0], 1, img_rows, img_cols)
        else:
            self.x_train = self.x_train.reshape(self.x_train.shape[0], img_rows, img_cols, 1)
            self.x_test = self.x_test.reshape(self.x_test.shape[0], img_rows, img_cols, 1)

        # convert class vectors to binary class matrices
        self.y_train = np.eye(num_classes)[self.y_train]
        self.y_test = np.eye(num_classes)[self.y_test]

Overwriting custom_data_handler.py


In [4]:
## Model, Dataset and Fusion Algorithm

components = dashboard_ui.generate_model_dataset_fusion_ui()

# GridBox layout for UI
grid = GridspecLayout(2,2)

grid[0,:] = GridBox(children=list(components[:-4]),
                    layout=Layout(
                        width='100%',
                        grid_template_rows='auto auto',
                        grid_template_columns='48% 48%',
                        grid_template_areas='''
                        "model_header model_header"
                        "model_dr model_upload"
                        "dataset_header dataset_header"
                        "dataset dataset_spl"
                        "ppp ppp"
                        '''
#                         ,border='0.5px solid black'
                    ))

grid[1,:] = GridBox(children=list(components[-4:]),
                    layout=Layout(
                        height='150px',
                        width='100%',
                        grid_template_rows='auto auto',
                        grid_template_columns='48% 48%',
                        grid_gap = '0px 0px',
                        grid_template_areas='''
                        "custom_data  custom_data_html"
                        "fusion_dr metrics_choice"
                        '''
#                         , border='0.5px solid black'
                    ))
# grid[2,:] = GridBox(children=list(components[-1:]),
#                     layout=Layout(
#                         height='55px',
#                         width='auto',
#                         grid_template_rows='100%',
#                         grid_template_columns='100%',
#                         grid_template_areas='''
#                         "fusion_dr"
#                         ''',
#                         border='0.5px solid black'
#                     ))
grid

GridspecLayout(children=(GridBox(children=(HTML(value='<h4>Model details', layout=Layout(grid_area='model_head…

### Choose number of parties and hyperparameters
Ensure you click `Confirm Hyperparameters` when done!

In [5]:
## Parties and Hyperparameters

components = list(dashboard_ui.generate_parties_hyperparams_ui())

# GridBox layout for UI
grid = GridspecLayout(2,3)

grid[0,:] = GridBox(children=components[:-2],
       layout = Layout(
           width='100%',
           grid_template_rows='auto auto',
           grid_template_columns='48% 48%',
           grid_template_areas='''
           "header_parties header_parties"
           "parties parties"
           "header_hyperparams header_hyperparams"
            ''')
       )
# Nested grid to vary spacing across various widgets
sub_grid_hyperparams = GridspecLayout(2,3)
sub_grid_hyperparams[0,:] = components[-1]
sub_grid_hyperparams[1,1] = components[-2]

grid[1, :] = sub_grid_hyperparams
party_hyperparam_ui = Output()

with party_hyperparam_ui:
    display(grid)
party_hyperparam_ui

Output()

#### Provide Party specific data files

- Only if you wish to use a Custom Dataset
- Chose Yes in the `Custom Dataset?` option in Step 1.2 above

In [7]:
## Upload party data files for each party:
if 'custom_data' in dashboard_ui.mgr.nb_config:
    upload_boxes = dashboard_ui.generate_custom_party_data_ui()
    for each in upload_boxes:
        display(each)

### Choose whether to run locally or on remote machines

In [8]:
print({
    "machines": {
        "venv_uses_conda": False,
        "venv_dir": "/home/trevormaxjs/Documents/Engineering/Research/venv",
        "machine1": {
            "ip_address": "127.0.0.1",
            "port_number": "5000",
            "ssh_username": "trevormaxjs",
            "staging_dir": "/home/trevormaxjs/Documents/Engineering/Research/",
            "ibmfl_dir": "/home/trevormaxjs/Documents/Engineering/Research/ibmfl"
        },
        "machine2": {
            "ip_address": "192.168.1.167",
            "port_number": "8102",
            "ssh_username": "trevormaxjs",
            "staging_dir": "/home/trevormaxjs/Documents/ibmfl",
            "ibmfl_dir": "/home/trevormaxjs/Documents/ibmfl/ibmfl"
        },
        "machine3": {
            "ip_address": "192.168.1.167",
            "port_number": "8103",
            "ssh_username": "trevormaxjs",
            "staging_dir": "/home/trevormaxjs/Documents/ibmfl/",
            "ibmfl_dir": "/home/trevormaxjs/Documents/ibmfl/ibmfl"
        }
    },
    "experiments": [{
    "agg_machine": "machine1",
    "local_staging_dir": "/home/trevormaxjs/Documents/Engineering/Research/",
}],
    "isLocalRun": False
})

{'machines': {'venv_uses_conda': False, 'venv_dir': '/home/trevormaxjs/Documents/Engineering/Research/venv', 'machine1': {'ip_address': '127.0.0.1', 'port_number': '5000', 'ssh_username': 'trevormaxjs', 'staging_dir': '/home/trevormaxjs/Documents/Engineering/Research/', 'ibmfl_dir': '/home/trevormaxjs/Documents/Engineering/Research/ibmfl'}, 'machine2': {'ip_address': '192.168.1.167', 'port_number': '8102', 'ssh_username': 'trevormaxjs', 'staging_dir': '/home/trevormaxjs/Documents/ibmfl', 'ibmfl_dir': '/home/trevormaxjs/Documents/ibmfl/ibmfl'}, 'machine3': {'ip_address': '192.168.1.167', 'port_number': '8103', 'ssh_username': 'trevormaxjs', 'staging_dir': '/home/trevormaxjs/Documents/ibmfl/', 'ibmfl_dir': '/home/trevormaxjs/Documents/ibmfl/ibmfl'}}, 'experiments': [{'agg_machine': 'machine1', 'local_staging_dir': '/home/trevormaxjs/Documents/Engineering/Research/'}], 'isLocalRun': False}


In [6]:
## Local or Remote run

components = dashboard_ui.generate_local_remote_ui()
# grid for displaying networking fields -- IP addr, port, ssh user, paths
partyDetails_grid = GridspecLayout(1,3)
partyDetails_grid[0, :] = components[1] # networking_deets_box 

display(components[0])
partyDetails_grid

Box(children=(HTML(value='<h4>Run this experiment locally or on remote machines?', layout=Layout(width='auto')…

GridspecLayout(children=(VBox(layout=Layout(grid_area='widget001')),), layout=Layout(grid_template_areas='"wid…

Incorrect JSON passed for remote details, check and retry!
Incorrect JSON passed for remote details, check and retry!
Incorrect JSON passed for remote details, check and retry!
Incorrect JSON passed for remote details, check and retry!
Incorrect JSON passed for remote details, check and retry!
Incorrect JSON passed for remote details, check and retry!
Incorrect JSON passed for remote details, check and retry!
Incorrect JSON passed for remote details, check and retry!


### Generate and View Aggregator and Party Config

In [7]:
## Generate Configs and Display them

components = dashboard_ui.generate_display_configs_ui()

# grid for displaying generated configurations
display_grid_1 = GridspecLayout(1,3)
display_grid_1[0, :] = components[1] # config_box

display_grid_1

Generating Data...
Data files saved to: /home/trevormaxjs/Documents/Engineering/Research//20230422T183302/data/mnist/random
Generating Configs...
Aggregator configs saved to: /home/trevormaxjs/Documents/Engineering/Research/20230422T183302/configs/coordinate_median/keras/config_agg.yml
Party configs saved to: /home/trevormaxjs/Documents/Engineering/Research/20230422T183302/configs/coordinate_median/keras/config_party*.yml
Updated Aggregator config at /home/trevormaxjs/Documents/Engineering/Research/20230422T183302/configs/coordinate_median/keras/config_agg.yml
Updated Party configs at /home/trevormaxjs/Documents/Engineering/Research/20230422T183302/configs/coordinate_median/keras/config_party*.yml


GridspecLayout(children=(VBox(children=(HTML(value='<h4>Configs Generated:', layout=Layout(width='auto')), HBo…

### Run the Experiment and Visualise Metrics
If the configs above look alright, go ahead and run the cell below to run the experiment!

In [ ]:
## Run the experiment and see charts

import ibmfl_cli_automator.run as ibmfl_runner
from ipywidgets import Button, VBox, Output

exp_runner = ibmfl_runner.Runner()

monitoring_box = VBox()

no_plots_for_these = [] # ['Federated Averaging', 'Gradient Averaging', 'Probabilistic Federated Neural Matching', 'Zeno', 'Shuffled Iterative Avg']

plot_button = Button(
        description='Show Charts',
        disabled=False,
        button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Displays the various plots for the experiment that ran',
        layout = Layout(width='120px', height='40px', margin='5px 50px 5px 400px') ## margin to position button centrally
    )


def invoke_runner():
    monitoring_out = Output(layout={'border': '0.5px solid black'})
    monitoring_box.children = [monitoring_out]
    display(display_grid_2)

    # some values needed by the Runner; there's only one trial for now
    dashboard_ui.mgr.run_details['experiments'][0]['shuffle_party_machines'] = False
    dashboard_ui.mgr.run_details['experiments'][0]['n_trials'] = 1
    dashboard_ui.mgr.run_details['experiments'][0]['n_parties'] = dashboard_ui.mgr.nb_config['global']['num_parties']
    dashboard_ui.mgr.run_details['experiments'][0]['n_rounds'] = dashboard_ui.mgr.nb_config['global']['rounds']

    # values for postprocessing and showing default metrics
    if dashboard_ui.mgr.nb_config['record_metrics']:
        dashboard_ui.mgr.run_details['experiments'][0]['postproc_fn'] = {}
        dashboard_ui.mgr.run_details['experiments'][0]['postproc_fn'] = 'gen_reward_vs_time_plots'
        dashboard_ui.mgr.run_details['experiments'][0]['postproc_x_key'] = 'post_train:ts'
        dashboard_ui.mgr.run_details['experiments'][0]['postproc_y_keys'] = ['post_train:eval:loss', 'post_train:eval:acc']
        # , 'post_train:eval:precision weighted', 'post_train:eval:recall weighted']

    exp_machines = exp_runner.convert_machine_dict_from_nb_to_cli(dashboard_ui.mgr.run_details['machines'])

    for exp_info in dashboard_ui.mgr.run_details['experiments']:
        with open('{}/config_agg.yml'.format(dashboard_ui.mgr.nb_config['local_conf_dir']), 'r') as config_agg_file:
            config_agg = config_agg_file.read()
        config_parties = []
        for pi in range(exp_info['n_parties']):
            with open('{}/config_party{}.yml'.format(dashboard_ui.mgr.nb_config['local_conf_dir'], pi), 'r') as config_party_file:
                config_parties += [config_party_file.read()]
        with monitoring_out:
            display(exp_runner.run_experiment(exp_info, dashboard_ui.mgr.run_details['machines'],
                                              config_agg, config_parties, ui_mode='nb', ts=dashboard_ui.mgr.nb_config['timestamp_str']) \
                    or 'Finished!')

    if dashboard_ui.mgr.nb_config['record_metrics']:
        if 'Keras' in dashboard_ui.mgr.nb_config['model'] and dashboard_ui.mgr.nb_config['fusion'] not in no_plots_for_these:
            # only some Keras models have plots currently
            monitoring_box.children = monitoring_box.children + (plot_button,)
        else:
            with monitoring_out:
                display('Plots for chosen model/fusion algorithm are not supported yet') # metrics processing not in place
    else:
        with monitoring_out:
            display('No metrics were recorded, so no plots to show')

plots_box = VBox()

def get_plots(b):
    b.disabled = True
    plots_out = Output(layout={'border': '0.5px solid black'})
    plots_box.children = [plots_out]
    display(display_grid_3)
    # generate the plot(s)
    with plots_out:
        display(exp_info = exp_runner.call_postproc_fn())

plot_button.on_click(get_plots)

# grid for displaying progress of running experiment
display_grid_2 = GridspecLayout(1,1)
display_grid_2[0, :] = monitoring_box

# grid for displaying charts from collected metrics
display_grid_3 = GridspecLayout(1,1)
display_grid_3[0, :] = plots_box

invoke_runner()

GridspecLayout(children=(VBox(children=(Output(layout=Layout(border='0.5px solid black')),), layout=Layout(gri…

Timeout - before timeout all parties can upload to aggregator, after they cannot.

Quorum - *minimum* percentage of parties the aggregator will wait for before aggregating. If less than minimum respond, we get a quorum exception.



Next steps - How to design experiments and calculate sensitivity w.r.t.:


- number of parties: could fix party dataset size or total dataset size

- number of rounds vs number of epochs

- try different models, keras vs pytorch

- sampling

- fusion algorithm: fed avg, iter avg, etc

meet 1:30 3/29

In [18]:
import yaml

In [19]:
with open("../../global_config.yaml", 'w') as out:
    yaml.safe_dump(dashboard_ui.mgr.run_details, out, default_flow_style=False)

In [13]:
dashboard_ui.mgr.run_details

{'machines': {'local0': {'ip_address': '127.0.0.1',
   'port_number': 5000,
   'ssh_username': 'trevormaxjs'},
  'local1': {'ip_address': '127.0.0.1',
   'port_number': 8085,
   'ssh_username': 'trevormaxjs'},
  'local2': {'ip_address': '127.0.0.1',
   'port_number': 8086,
   'ssh_username': 'trevormaxjs'},
  'local3': {'ip_address': '127.0.0.1',
   'port_number': 8087,
   'ssh_username': 'trevormaxjs'},
  'local4': {'ip_address': '127.0.0.1',
   'port_number': 8088,
   'ssh_username': 'trevormaxjs'},
  'local5': {'ip_address': '127.0.0.1',
   'port_number': 8089,
   'ssh_username': 'trevormaxjs'},
  'local6': {'ip_address': '127.0.0.1',
   'port_number': 8090,
   'ssh_username': 'trevormaxjs'},
  'local7': {'ip_address': '127.0.0.1',
   'port_number': 8091,
   'ssh_username': 'trevormaxjs'},
  'local8': {'ip_address': '127.0.0.1',
   'port_number': 8092,
   'ssh_username': 'trevormaxjs'},
  'local9': {'ip_address': '127.0.0.1',
   'port_number': 8093,
   'ssh_username': 'trevormaxjs'}